# Week 4 Notebook: Data Preprocessing
The goal of this week's assignment is to continue to preprocess our data by cleaning it, treating issues such as outliers and missing values, transforming variables, and making the data model-ready. 

In [10]:
### Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
from geopy.geocoders import Nominatim

### Load the dataframe

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

data_folder = os.path.join(parent_dir,"data")
raw_data_folder = os.path.join(data_folder,"raw")
interim_data_folder = os.path.join(data_folder,"interim")

uber_file_path = os.path.join(raw_data_folder, "uber.csv.zip")
lyft_file_path = os.path.join(raw_data_folder, "lyft.csv.zip")

In [3]:

if os.path.exists(uber_file_path):
    with zipfile.ZipFile(uber_file_path, 'r') as zip_ref:
        zip_ref.extractall(raw_data_folder)
    print(f"Uber file extracted to: {raw_data_folder}")
else:
    print(f"Uber file not found: {uber_file_path}")

if os.path.exists(lyft_file_path):
    with zipfile.ZipFile(lyft_file_path, 'r') as zip_ref:
        zip_ref.extractall(raw_data_folder)
    print(f"Lyft file extracted to: {raw_data_folder}")
else:
    print(f"Lyft file not found: {lyft_file_path}")

Uber file extracted to: /Users/carriexia/Documents/GitHub/ADAN8888.01_Fall_24_Applied_Analytics_Project/data/raw
Lyft file extracted to: /Users/carriexia/Documents/GitHub/ADAN8888.01_Fall_24_Applied_Analytics_Project/data/raw


In [4]:
uber_csv_path = os.path.join(raw_data_folder, "uber.csv")
lyft_csv_path = os.path.join(raw_data_folder, "lyft.csv")

uber_df = pd.read_csv(uber_csv_path)
lyft_df = pd.read_csv(lyft_csv_path)
    
df = pd.concat([uber_df, lyft_df], ignore_index=True)

### Split Train Test Validation

In [5]:
def train_val_test_split(df):
    # Suffle the dataset and calculate the size of validation and test sets

    df = df.sample(frac=1, random_state=123)

    val_size = int(len(df) * 0.2)
    test_size = int(len(df) * 0.1)

    # Select rows based on the val_size and test_size to store as train set, val set, and test set
    train_df = df.iloc[val_size + test_size:]
    val_df = df.iloc[:val_size]
    test_df = df.iloc[val_size:val_size + test_size]
    return train_df, val_df, test_df

train_df, val_df, test_df = train_val_test_split(df)

## Missing Value Imputation
Uber Taxi Fee Breakdown
 - base fare: $2.60 for first 1/7 mile
 - per minute fare: $0.47
 - per mile: $2.8


In [6]:
def taxi_price_calculator(distance, time):
    base_fare = 2.60
    per_min_fare = 0.47
    per_mile_fare = 2.8
    price = base_fare + distance * per_mile_fare + time * per_min_fare
    return price

Since we only have one time stamp rather the duration of each ride. We will need to figure out a way to estimate the time costed for each Taxi ride for a more accurate imputation. Here are the Steps.

1. Get the unique records for locations as a lst and save the unique combination of sources as a csv
2. Get the longtitude and latitude for these loctaions and save it as a dict with location as the key and value being [lat, long]
3. Read the csv and create sourece latitude, soure longtitude, desitination latitude, destination longtitude columns based on the location dict
4. Use the Mapbox Direction API to pull time estimated by driving
5. Save the csv with source, destination, time_estimated

Please refer to the code folder to get the code for how we get the estimated duration for the taxi rides.

In [7]:
unique_combos = df[["source","destination"]].drop_duplicates()
unique_combos_dir = os.path.join(interim_data_folder, "unique_combo.csv")
unique_combos.to_csv(unique_combos_dir)

locations = pd.unique(df[["source","destination"]].values.ravel())
print(locations)

['North End' 'West End' 'Beacon Hill' 'South Station' 'North Station'
 'Fenway' 'Boston University' 'Back Bay' 'Theatre District'
 'Northeastern University' 'Financial District' 'Haymarket Square']


In [11]:
def get_longtitude_latitude(location):
    # calling the Nominatim tool and create Nominatim class
    loc = Nominatim(user_agent="Geopy Library")
    location = location+", Boston"
    getLoc = loc.geocode(location)
    return getLoc.latitude, getLoc.longitude

location_dict = {}

for location in locations:
    lat, long = get_longtitude_latitude(location)
    if lat is not None and long is not None:
        location_dict[location] = [lat, long]

In [12]:
unique_combos["source_lat"] = unique_combos['source'].apply(lambda x: location_dict[x][0])
unique_combos["source_long"] = unique_combos['source'].apply(lambda x: location_dict[x][1])

unique_combos["destination_lat"] = unique_combos['destination'].apply(lambda x: location_dict[x][0])
unique_combos["destination_long"] = unique_combos['destination'].apply(lambda x: location_dict[x][1])

ride_locations_dir = os.path.join(interim_data_folder, "ride_locations.csv")
df.to_csv(ride_locations_dir, index = False)

The  `get_eta` function requires api_key, so I saved the dataframe locally for reviewing convinience. Pleaser refer to the `time_calulator.py` under the codes folder for more details how the time was imputed.

In [13]:

rides_with_eta = os.path.join(interim_data_folder, "rides_with_etas.csv")
time_df = pd.read_csv(rides_with_eta)
time_df.head()

,source,destination,eta_minutes
0,North End,West End,7.751050
1,Beacon Hill,South Station,10.896667
2,North Station,Fenway,13.937950
3,North End,Beacon Hill,12.600567
4,Boston University,North Station,15.743100


In [14]:
df = pd.merge(df, time_df[['source', 'destination', 'eta_minutes']], on=['source', 'destination'], how = 'left')

In [15]:
df.head()

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime,eta_minutes
0,009e9c53-074d-43cf-aef2-0fbc7a47ed3d,1.543616e+09,22,30,11,2018-11-30 22:13:01,America/New_York,North End,West End,Uber,...,1543593600,28.79,1543579200,42.52,1543600800,26.41,1543575600,40.53,1543611600,7.75105
1,23f145da-f0c1-4d1f-a184-496bc003a7db,1.544698e+09,10,13,12,2018-12-13 10:50:11,America/New_York,North End,West End,Uber,...,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800,7.75105
2,357559cb-8c58-4278-a41a-e33b2e0997a3,1.544729e+09,19,13,12,2018-12-13 19:15:03,America/New_York,North End,West End,Uber,...,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800,7.75105
3,50ef1165-9d23-416c-a65c-18906207b295,1.545005e+09,23,16,12,2018-12-16 23:55:11,America/New_York,North End,West End,Uber,...,1544979600,39.22,1544954400,43.83,1544990400,33.98,1545019200,38.38,1544986800,7.75105
4,91c4861c-1780-42b0-bca1-bbd64a422cc3,1.544748e+09,0,14,12,2018-12-14 00:40:07,America/New_York,North End,West End,Uber,...,1544716800,18.29,1544688000,33.83,1544731200,13.79,1544688000,32.85,1544734800,7.75105


In [16]:
df.loc[df['name'] == 'Taxi', 'price'] = df.loc[df['name'] == 'Taxi'].apply(
    lambda row: taxi_price_calculator(row['distance'], row['eta_minutes']), axis=1)

In [17]:
df_taxi = df[df["name" ]== "Taxi"]
df_taxi.isnull().sum()

id                             0
timestamp                      0
hour                           0
day                            0
month                          0
datetime                       0
timezone                       0
source                         0
destination                    0
cab_type                       0
product_id                     0
name                           0
price                          0
distance                       0
surge_multiplier               0
latitude                       0
longitude                      0
temperature                    0
apparentTemperature            0
short_summary                  0
long_summary                   0
precipIntensity                0
precipProbability              0
humidity                       0
windSpeed                      0
windGust                       0
windGustTime                   0
visibility                     0
temperatureHigh                0
temperatureHighTime            0
temperatur

In [18]:
df = df.drop('eta_minutes', axis = 1)

In [19]:
df_na = df[df['price'].isna()]
missing_percentage_after_imputing = df['price'].isna().sum() / len(df) * 100
print(f"Percentage of missing values in 'price' after imputing: {missing_percentage_after_imputing:.2f}%")

Percentage of missing values in 'price' after imputing: 0.00%
